# Wind Statistics

### Introduction:

The data have been modified to contain some missing values, identified by NaN.  
Using pandas should make this exercise
easier, in particular for the bonus question.

You should be able to perform all of these operations without using
a for loop or other looping construct.


1. The data in 'wind.data' has the following format:

In [434]:
"""
Yr Mo Dy   RPT   VAL   ROS   KIL   SHA   BIR   DUB   CLA   MUL   CLO   BEL   MAL
61  1  1 15.04 14.96 13.17  9.29   NaN  9.87 13.67 10.25 10.83 12.58 18.50 15.04
61  1  2 14.71   NaN 10.83  6.50 12.62  7.67 11.50 10.04  9.79  9.67 17.54 13.83
61  1  3 18.50 16.88 12.33 10.13 11.17  6.17 11.25   NaN  8.50  7.67 12.75 12.71
"""

'\nYr Mo Dy   RPT   VAL   ROS   KIL   SHA   BIR   DUB   CLA   MUL   CLO   BEL   MAL\n61  1  1 15.04 14.96 13.17  9.29   NaN  9.87 13.67 10.25 10.83 12.58 18.50 15.04\n61  1  2 14.71   NaN 10.83  6.50 12.62  7.67 11.50 10.04  9.79  9.67 17.54 13.83\n61  1  3 18.50 16.88 12.33 10.13 11.17  6.17 11.25   NaN  8.50  7.67 12.75 12.71\n'

   The first three columns are year, month and day.  The
   remaining 12 columns are average windspeeds in knots at 12
   locations in Ireland on that day.   

   More information about the dataset go [here](wind.desc).

### Step 1. Import the necessary libraries

In [1]:
import numpy as np
import pandas as pd

### Step 2. Import the dataset from this [address](https://github.com/guipsamora/pandas_exercises/blob/master/Stats/Wind_Stats/wind.data)

### Step 3. Assign it to a variable called data and replace the first 3 columns by a proper datetime index.

In [52]:
data = pd.read_csv('wind.data', sep='\s+')
data["Period"] = data["Yr"].map(str) + "-" + data["Mo"].map(str) + "-" + data["Dy"].map(str)
data["Period"] = pd.to_datetime(data["Period"])
data = data.loc[:, "RPT":"Period"].set_index("Period")


### Step 4. Year 2061? Do we really have data from this year? Create a function to fix it and apply it.

In [53]:
data.index = data.index.map(lambda x: x - pd.DateOffset(years = 100))

### Step 5. Set the right dates as the index. Pay attention at the data type, it should be datetime64[ns].

In [54]:
data.index.dtype_str

'datetime64[ns]'

### Step 6. Compute how many values are missing for each location over the entire record.  
#### They should be ignored in all calculations below. 

In [55]:
np.count_nonzero(data.isnull())

31

### Step 7. Compute how many non-missing values there are in total.

In [56]:
np.count_nonzero(~data.isnull())

78857

### Step 8. Calculate the mean windspeeds of the windspeeds over all the locations and all the times.
#### A single number for the entire dataset.

In [59]:
data.fillna(value = 0, inplace=True)
np.mean(np.mean(data))

10.223864592840467

### Step 9. Create a DataFrame called loc_stats and calculate the min, max and mean windspeeds and standard deviations of the windspeeds at each location over all the days 

#### A different set of numbers for each location.

In [79]:
loc_stats = pd.DataFrame([np.min(data), np.max(data), np.mean(data), np.std(data)]).transpose()
loc_stats.columns = ['Min', 'Max', 'Mean', 'Std']
data.head()


,RPT,VAL,ROS,KIL,SHA,BIR,DUB,CLA,MUL,CLO,BEL,MAL
Period,,,,,,,,,,,,
1961-01-01,15.04,14.96,13.17,9.29,0.00,9.87,13.67,10.25,10.83,12.58,18.50,15.04
1961-01-02,14.71,0.00,10.83,6.50,12.62,7.67,11.50,10.04,9.79,9.67,17.54,13.83
1961-01-03,18.50,16.88,12.33,10.13,11.17,6.17,11.25,0.00,8.50,7.67,12.75,12.71
1961-01-04,10.58,6.63,11.75,4.58,4.54,2.88,8.63,1.79,5.83,5.88,5.46,10.88
1961-01-05,13.33,13.25,11.42,6.17,10.71,8.21,11.92,6.54,10.92,10.34,12.92,11.83


### Step 10. Create a DataFrame called day_stats and calculate the min, max and mean windspeed and standard deviations of the windspeeds across all the locations at each day.

#### A different set of numbers for each day.

In [75]:
def series_for(fn):
    return data.apply(fn, axis = 1)


data_stats = pd.DataFrame([series_for(np.min), series_for(np.max), series_for(np.mean), series_for(np.std)]).transpose()
data_stats.columns = ['Min', 'Max', 'Mean', 'Std']
data_stats.head()

,Min,Max,Mean,Std
Period,,,,
1961-01-01,0.00,18.50,11.933333,4.418220
1961-01-02,0.00,17.54,10.391667,4.276881
1961-01-03,0.00,18.50,10.671667,4.652977
1961-01-04,1.79,11.75,6.619167,3.061972
1961-01-05,6.17,13.33,10.630000,2.341250


### Step 11. Find the average windspeed in January for each location.  
#### Treat January 1961 and January 1962 both as January.

In [84]:
jan = data.index.map(lambda x: x.month) == 1
jandata = data[jan]
jandata.apply(np.mean, axis = 0)

RPT    14.820717
VAL    12.891416
ROS    13.299624
KIL     7.199498
SHA    11.625914
BIR     8.054839
DUB    11.819355
CLA     9.495000
MUL     9.543208
CLO    10.053566
BEL    14.550520
MAL    18.028763
dtype: float64

### Step 12. Downsample the record to a yearly frequency for each location.

### Step 13. Downsample the record to a monthly frequency for each location.

### Step 14. Downsample the record to a weekly frequency for each location.

### Step 15. Calculate the mean windspeed for each month in the dataset.  
#### Treat January 1961 and January 1962 as *different* months.
#### (hint: first find a  way to create an identifier unique for each month.)

### Step 16. Calculate the min, max and mean windspeeds and standard deviations of the windspeeds across all locations for each week (assume that the first week starts on January 2 1961) for the first 52 weeks.